In [1]:
# Import required libraries and dependencies
import warnings
warnings.filterwarnings("ignore", category=UserWarning)  # Stop KMeans memory leak warning

import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# For 3D plots at the end
import matplotlib.pyplot as plt

In [2]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384
binancecoin,2.10423,12.85511,6.80688,0.05865,36.33486,155.61937,69.69195
chainlink,-0.23935,20.69459,9.30098,-11.21747,-43.69522,403.22917,325.13186
cardano,0.00322,13.99302,5.55476,10.10553,-22.84776,264.51418,156.09756
litecoin,-0.06341,6.60221,7.28931,1.21662,-17.23960,27.49919,-12.66408


In [3]:
# Generate summary statistics
df_market_data.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [4]:
df_market_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 41 entries, bitcoin to digibyte
Data columns (total 7 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   price_change_percentage_24h   41 non-null     float64
 1   price_change_percentage_7d    41 non-null     float64
 2   price_change_percentage_14d   41 non-null     float64
 3   price_change_percentage_30d   41 non-null     float64
 4   price_change_percentage_60d   41 non-null     float64
 5   price_change_percentage_200d  41 non-null     float64
 6   price_change_percentage_1y    41 non-null     float64
dtypes: float64(7)
memory usage: 2.6+ KB


In [5]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=850,
    height=450,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

---

### Prepare the Data

In [6]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
scaled_array = StandardScaler().fit_transform(df_market_data)
print(scaled_array[0:5])

[[ 0.50852937  0.49319307  0.77220043  0.23545963 -0.0674951  -0.35595348
  -0.25163688]
 [ 0.18544589  0.93444504  0.55869212 -0.05434093 -0.27348273 -0.11575947
  -0.19935211]
 [ 0.02177396 -0.70633685 -0.02168042 -0.06103015  0.00800452 -0.55024692
  -0.28206051]
 [-0.04076438 -0.81092807  0.24945797 -0.05038797 -0.37316402 -0.45825882
  -0.29554614]
 [ 1.19303608  2.00095907  1.76061001  0.54584206 -0.29120287 -0.49984776
  -0.27031695]]


In [7]:
# Create a DataFrame with the scaled data
df_cryptscale_origin = pd.DataFrame(scaled_array, index=df_market_data.index, columns=df_market_data.columns)

# Copy the crypto names from the original DataFrame
coins = list(df_market_data.index)

# Set the coin_id column as index
df_cryptscale_origin['coin_id'] = coins
crypto_scaled = df_cryptscale_origin.set_index('coin_id')

# Display the scaled DataFrame
crypto_scaled.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317


---

### Find the Best Value for k Using the Original Scaled DataFrame.

In [8]:
# Create a list with the number of k-values from 1 to 11
k = list(range(1, 12))
k

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [9]:
# Create an empty list to store the inertia values
inertia = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
for i in k:
# 1. Create a KMeans model using the loop counter for the n_clusters
    model = KMeans(n_clusters=i, random_state=42, n_init="auto")
# 2. Fit the model to the data using `df_market_data_scaled`
    model.fit(crypto_scaled)
# 3. Append the model.inertia_ to the inertia list
    inertia.append(model.inertia_)

In [10]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {
        "k": k,
        "inertia": inertia
    }


# Create a DataFrame with the data to plot the Elbow curve
df_elbow = pd.DataFrame(elbow_data)

In [11]:
# Plot a line chart with all the inertia values computed with
# the different values of k to visually identify the optimal value for k.
    # Plot a line chart with all the inertia values computed with 
    # the different values of k to visually identify the optimal value for k.
df_elbow.hvplot.line(
    x="k",
    y="inertia",
    title= "Elbow Curve",
    xticks=k
)

:Curve   [k]   (inertia)

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** k = 4

---

### Cluster Cryptocurrencies with K-means Using the Original Scaled DataFrame

In [12]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=4, random_state=1)

In [13]:
# Fit the K-Means model using the scaled DataFrame
model.fit(crypto_scaled)

KMeans(n_clusters=4, random_state=1)

In [14]:
# Predict the clusters to group the cryptocurrencies using the scaled DataFrame
k_4 = model.predict(crypto_scaled)


# Print the resulting array of cluster values.
k_4

array([2, 2, 0, 0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 2, 0, 2, 0, 0, 2, 0, 0, 2,
       0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 3, 2, 0, 0, 1, 0, 0, 0, 0])

In [15]:
# Create a copy of the scaled DataFrame
scaled_copy = crypto_scaled.copy()

In [16]:
# Add a new column to the copy of the scaled DataFrame with the predicted clusters
scaled_copy['cluster_predict'] = k_4

# Display the copy of the scaled DataFrame
scaled_copy.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y,cluster_predict
coin_id,,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637,2
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352,2
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061,0
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546,0
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317,2


In [17]:

# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
scaled_copy.hvplot.scatter(
    x='price_change_percentage_24h',
    y='price_change_percentage_7d',
    by='cluster_predict',
    hover_cols = 'coin_id'
)

:NdOverlay   [cluster_predict]
   :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)

---

### Optimize Clusters with Principal Component Analysis.

In [18]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components=3)

In [19]:
# Use the PCA model with `fit_transform` to reduce the original scaled DataFrame
# down to three principal components.
clusters_pca=pca.fit_transform(scaled_copy)

# View the scaled PCA data
clusters_pca[:5]


array([[ 0.44890795,  1.24537573,  0.8506404 ],
       [ 0.49536709,  0.899823  ,  1.31755943],
       [-0.81884571, -0.07189909, -0.69501507],
       [-0.84035711, -0.08005407, -0.54435952],
       [ 0.81324004,  2.66952035,  1.64332113]])

In [20]:
# Retrieve the explained variance to determine how much information
# can be attributed to each principal component.
pca.explained_variance_ratio_


array([0.34871677, 0.31363391, 0.22627118])

In [21]:
#**Question:** What is the total explained variance of the three principal components?
sum(pca.explained_variance_ratio_)

0.8886218549859446

#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** 0.8886218549859446

In [22]:
# Create a new DataFrame with the PCA data.
clusters_pca_df = pd.DataFrame(
    clusters_pca,
    columns=["PCA1", "PCA2", "PCA3"]
)
clusters_pca_df

# Copy the crypto names from the original scaled DataFrame
clusters_pca_df['coin_id'] = list(scaled_copy.index)
clusters_pca_df

# Set the coin_id column as index
clusters_pca_df = clusters_pca_df.set_index('coin_id')

# Display the scaled PCA DataFrame
clusters_pca_df

,PCA1,PCA2,PCA3
coin_id,,,
bitcoin,0.448908,1.245376,0.850640
ethereum,0.495367,0.899823,1.317559
tether,-0.818846,-0.071899,-0.695015
ripple,-0.840357,-0.080054,-0.544360
bitcoin-cash,0.813240,2.669520,1.643321
binancecoin,0.822871,1.682096,0.905203
chainlink,0.913868,1.128684,2.730038
cardano,0.811102,1.113378,1.598655
litecoin,0.258728,0.946884,1.103560


---

### Find the Best Value for k Using the Scaled PCA DataFrame

In [34]:
# Create a list with the number of k-values from 1 to 11
k = list(range(1,12))
k

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [35]:
# Create an empty list to store the inertia values
intertia2=[]

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_pca`
# 3. Append the model.inertia_ to the inertia list
for i in k:
    k_model=KMeans(n_clusters=i, random_state=42)
    k_model.fit(clusters_pca_df)
    intertia2.append(k_model.inertia_)

In [36]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data2={'k': k, 'inertia':intertia2}
# Create a DataFrame with the data to plot the Elbow curve
elbow_df2=pd.DataFrame(elbow_data2)
elbow_df2

,k,inertia
0,1,290.622694
1,2,203.579726
2,3,199.647134
3,4,108.031002
4,5,34.461155
5,6,28.395881
6,7,17.849218
7,8,15.273560
8,9,12.288515
9,10,9.976910


In [37]:
# Plot a line chart with all the inertia values computed with
# the different values of k to visually identify the optimal value for k.
elbow_df2.hvplot.line(
    x="k", 
    y="inertia", 
    title="Elbow Curve2", 
    xticks=k
)

:Curve   [k]   (inertia)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** It looks like the best fit for K using the PCA data is 4


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** No, using the original (scaled) data, based on the first Elbow Curve, also fit best with k=4.

### Cluster Cryptocurrencies with K-means Using the Scaled PCA DataFrame

In [38]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=4, random_state=42)

In [39]:
# Fit the K-Means model using the PCA data
model.fit(clusters_pca_df)

KMeans(n_clusters=4, random_state=42)

In [40]:
# Predict the clusters to group the cryptocurrencies using the scaled PCA DataFrame
k_4=model.predict(clusters_pca_df)

# Print the resulting array of cluster values.
k_4

array([0, 0, 2, 2, 0, 0, 0, 0, 0, 2, 1, 2, 2, 0, 2, 0, 1, 2, 0, 2, 1, 0,
       2, 2, 2, 2, 2, 1, 0, 2, 2, 2, 3, 0, 1, 2, 2, 1, 2, 2, 2])

In [41]:
# Create a copy of the scaled PCA DataFrame
copy_clusters_pca_df = clusters_pca_df.copy()

# Add a new column to the copy of the PCA DataFrame with the predicted clusters
copy_clusters_pca_df['predicted_cluster'] = k_4

# Display the copy of the scaled PCA DataFrame
copy_clusters_pca_df

,PCA1,PCA2,PCA3,predicted_cluster
coin_id,,,,
bitcoin,0.448908,1.245376,0.850640,0
ethereum,0.495367,0.899823,1.317559,0
tether,-0.818846,-0.071899,-0.695015,2
ripple,-0.840357,-0.080054,-0.544360,2
bitcoin-cash,0.813240,2.669520,1.643321,0
binancecoin,0.822871,1.682096,0.905203,0
chainlink,0.913868,1.128684,2.730038,0
cardano,0.811102,1.113378,1.598655,0
litecoin,0.258728,0.946884,1.103560,0


In [46]:
# Create a scatter plot using hvPlot by setting
# `x="PC1"` and `y="PC2"`.
# Color the graph points with the labels found using K-Means and
# add the crypto name in the `hover_cols` parameter to identify
# the cryptocurrency represented by each data point.
copy_clusters_pca_df.hvplot.scatter(
    x="PCA1",
    y="PCA2",
    by = 'predicted_cluster',
    hover_cols='coin_id'
)

:NdOverlay   [predicted_cluster]
   :Scatter   [PCA1]   (PCA2,coin_id)

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [47]:
# Composite plot to contrast the Elbow curves
df_elbow.hvplot.line(x="k", y="inertia", title= "Elbow Curve 1", xticks=k) + elbow_df2.hvplot.line(x="k", y="inertia", title="Elbow Curve 2", xticks=k)


:Layout
   .Curve.I  :Curve   [k]   (inertia)
   .Curve.II :Curve   [k]   (inertia)

In [48]:
# Composite plot to contrast the clusters
scaled_copy.hvplot.scatter( x='price_change_percentage_24h', y='price_change_percentage_7d', by='cluster_predict', hover_cols = 'coin_id') + copy_clusters_pca_df.hvplot.scatter(x="PCA1", y="PCA2", by = 'predicted_cluster', hover_cols='coin_id')


:Layout
   .NdOverlay.I  :NdOverlay   [cluster_predict]
      :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)
   .NdOverlay.II :NdOverlay   [predicted_cluster]
      :Scatter   [PCA1]   (PCA2,coin_id)

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** The K-means method showed more tightly grouped clusters due to less overall variance in values, while PCA created wider spacing between clusters with less overlap between outliers. For example, in K-means, "celsius_degree_token" and "digibyte" are in clusters 1 and 0 but appear close together, while PCA spaces them further apart, making their different clusters more logical.